<a href="https://colab.research.google.com/github/dookda/py499/blob/main/python_15_Raster_Geospatial_on_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api

In [ ]:
import ee

# Authenticate the Earth Engine library.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


In [ ]:
import ee
import folium
from folium.plugins import MarkerCluster
import geemap.core as geemap

m = geemap.Map()
m.set_center(99, 18.80, 8)

In [ ]:
roi = ee.Geometry.Polygon(
    [[[99.0, 18.0],
      [99.0, 19.0],
      [100.0, 19.0],
      [100.0, 18.0],
      [99.0, 18.0]]]
)

print(roi.getInfo())

In [ ]:
# add to feature collection
fc = ee.FeatureCollection([
    ee.Feature(roi)
])

# add name to property
fc = fc.set('name', 'Chiang Mai')

print(fc.getInfo())

m.addLayer(fc, {}, 'roi')

In [ ]:
ls9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
            .filterDate('2023-11-01', '2024-01-31') \
            .filterBounds(roi)

# try to filter by roi

In [ ]:
# Scale factors for radiance data
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

# apply scale factors to the dataset
dataset = ls9.map(apply_scale_factors)

In [ ]:
# clip the image with roi
dataset = dataset.map(lambda image: image.clip(roi))

In [ ]:
# True color visualization
vis_true = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

# False color visualization
vis_false = {
    'bands': ['SR_B5', 'SR_B4', 'SR_B3'],
    'min': 0.0,
    'max': 0.3,
}

In [ ]:
# add to map
m.add_layer(dataset, vis_true, 'True Color (432)')

In [ ]:
# reduces an image collectionmedian to mean, max, min 
median = dataset.median()
mean = dataset.mean()
max = dataset.max()
min = dataset.min()

# try to add to map
m.add_layer(median, vis_true, 'Median')
m.add_layer(min, vis_true, 'min')

In [ ]:

m.addLayer(dataset, vis_false, 'False color image')

In [ ]:
# NDVI visualization
def addNDVI(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

dataset = dataset.map(addNDVI)
ndvi = dataset.select('NDVI')
ndviParams = {'palette': ['blue', 'white', 'green']}
m.addLayer(ndvi, ndviParams, 'NDVI')

In [ ]:
# lst from modis
modis = ee.ImageCollection('MODIS/061/MOD11A1') \
    .filterDate('2023-11-01', '2024-03-31') \
    .select('LST_Day_1km') \
    .map(lambda image: image.clip(roi))

# kavin to celsius
cs = modis.map(lambda image: image.multiply(0.02).subtract(273.15))

print(cs.getInfo())

# visualization parameters
lstParams = {
    min: 35.0,
    max: 40.0,
    'palette':['blue', 'white', 'green'],
}
# add to map
m.addLayer(cs, lstParams, 'MODIS LST')



In [ ]:
# show the map
m